In [1]:
{-# LANGUAGE ScopedTypeVariables, TypeOperators #-}
import Prelude ()
import Control.Category.Constrained.Prelude
import Control.Arrow.Constrained

import Data.Manifold
import Data.Manifold.TreeCover
import Data.Manifold.Web
import Data.Manifold.Shade
import Data.Manifold.Types
import Data.Function.Affine
import Math.LinearMap.Category
import Math.LinearMap.Category.Derivatives
import Data.VectorSpace
import Linear (V2(..), _x, _y)
import Control.Lens

In [2]:
test x = x`seq`putStrLn "Ok"

In [3]:
import Graphics.Dynamic.Plot.R2
import Data.Colour.Names
import Data.Colour.Manifold

colourscheme :: Shade' ℝ -> Shade (Colour ℝ)
colourscheme = f `seq` \(Shade' y e) -> f (Shade y $ dualNorm e)
 where Just (f :: Shade ℝ -> Shade (Colour ℝ))
          = rangeWithinVertices (1, maxc)
                              [ (0, black) ]
       Just maxc = toInterior (grey :: Colour ℝ)
test colourscheme

colourscheme_nabla :: Shade' (V2 ℝ+>ℝ) -> Shade (Colour ℝ)
colourscheme_nabla = f `seq` \(Shade' y e) -> f (Shade y $ dualNorm e)
 where Just (f :: Shade (V2 ℝ+>ℝ) -> Shade (Colour ℝ))
          = rangeWithinVertices (zeroV, neutral)
                              [ (V2 2 0-+|>1, red)
                              , (V2 0 2-+|>1, green) ]
       Just neutral = toInterior (grey :: Colour ℝ)
test colourscheme_nabla

Ok

Ok

In [4]:
gaußianPeak :: [ℝ²] -> ℝ -> PointsWeb ℝ² (Shade' ℝ)
gaußianPeak ps δy = fromWebNodes euclideanMetric
       [ (xy, exp (-2*magnitudeSq xy) |±|[δy]) | xy <- ps ]

In [5]:
osc :: PointsWeb ℝ (Shade' ℝ)
osc = fromWebNodes euclideanMetric
           $ [(x, sin x|±|[0.001]) | x<-[0,0.2 .. pi]]
fst <$> localModels_BGrid osc

[3.100000000000001,2.900000000000001,2.7000000000000006,2.5000000000000004,2.3000000000000003,2.1,1.9,1.7000000000000002,0.9000000000000001,0.7000000000000002,1.1,1.3000000000000003,1.5,0.5,0.30000000000000004,0.1]

In [6]:
plotWindow [ plot osc  & legendName "sin"
           , plot (linIsoTransformShade (lfun ($ 1)) <$> differentiateUncertainWebFunction osc)
                       & legendName "sin' (nodes)"
           , plot (linIsoTransformShade (lfun ($ squareV 1)) <$> differentiate²UncertainWebFunction osc)
                       & legendName "sin'' (nodes)"
           , plot (fromWebNodes euclideanMetric
               $ second (linIsoTransformShade (lfun ($ 1)) . fst . snd . quadraticModel_decomposition)
                              <$> localModels_BGrid osc )
                       & legendName "sin' (links)"
           , plot (fromWebNodes euclideanMetric
               $ second (linIsoTransformShade (lfun ($ squareV 1)) . snd . snd . quadraticModel_decomposition)
                              <$> localModels_BGrid osc )
                       & legendName "sin'' (links)"
           , continFnPlot (negate . sin)
                       & legendName "-sin"
           ]

GraphWindowSpecR2{lBound=-0.5333333333333334, rBound=3.733333333333335, bBound=-1.3303114841836925, tBound=1.332414329787962, xResolution=640, yResolution=480}

![Numerically differentiating a function sampled on a 1D  web](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/WebSampling/1D-differentiation.png)

In [7]:
δ = 0.01
grid = [V2 x y | x<-[-2,-1.8..2], y<-[-1,-0.8..2]]

plotWindow [plot . fmap colourscheme $ gaußianPeak grid δ ]
plotWindow [plot . fmap colourscheme_nabla . differentiateUncertainWebFunction
              $ gaußianPeak grid δ ]
plotWindow [plot . fmap (colourscheme_nabla . projectShade (lensEmbedding (1*∂id/∂_x)))
   . differentiateUncertainWebFunction . differentiateUncertainWebFunction
       $ gaußianPeak grid δ ]
plotWindow [plot . fmap (colourscheme_nabla . projectShade (lensEmbedding ((1*∂id/∂_x)/∂id)))
   . differentiateUncertainWebFunction . differentiateUncertainWebFunction
       $ gaußianPeak grid δ ]

GraphWindowSpecR2{lBound=-2.666666666666667, rBound=2.6666666666666656, bBound=-1.5, tBound=2.4999999999999987, xResolution=640, yResolution=480}

GraphWindowSpecR2{lBound=-2.666666666666667, rBound=2.6666666666666656, bBound=-1.5000000000000002, tBound=2.499999999999999, xResolution=640, yResolution=480}

GraphWindowSpecR2{lBound=-2.6666666666666656, rBound=2.6666666666666656, bBound=-1.4999999999999996, tBound=2.4999999999999996, xResolution=640, yResolution=480}

GraphWindowSpecR2{lBound=-2.666666666666666, rBound=2.666666666666666, bBound=-1.4999999999999998, tBound=2.499999999999999, xResolution=640, yResolution=480}